In [ ]:
using Pkg
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="boozer",        # jac_type
    grid_type="ldp",
    mpsi = 128,
    mtheta = 256,
    psilow=0.01,             # psilow
    psihigh=0.994)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

In [ ]:
plasma_eq.sq.nqty

In [ ]:
fieldnames(typeof(plasma_eq.rzphi))

In [ ]:
plasma_eq.rzphi.fs[4]

In [ ]:
typeof(plasma_eq.rzphi.fs[1])

### CAlc. Metric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)


    print(JPEC.Spl.bicube_eval(plasma_eq.rzphi,0.5,0.01,0))
    # Metric tensor calculation
    metric_result = fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=0,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    println(plasma_eq.rzphi.ys)

    println(JPEC.Spl.bicube_eval(plasma_eq.rzphi,0.5,0.01,0))

    
    # Evaluate at a specific point (if spline is available)
    if metric_result.fspline !== nothing
        psi_test = 0.5
        theta_test = 0.0
        
        println(JPEC.Spl.fspline_eval(  metric_result.fspline,psi_test,theta_test,0))


    end
    


### Calc. Metric2

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric

try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=20,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    
    # 🔧 Define test coordinates FIRST
    psi_test = 0.5
    theta_test = 0*π/2
    
    println("\n🔍 Testing at coordinates: ψ=$psi_test, θ=$(round(theta_test/π, digits=2))π")

    # Test rzphi evaluation

        # Test rzphi evaluation
# Test rzphi evaluation
try
    rzphi_result = JPEC.SplinesMod.bicube_eval(plasma_eq.rzphi, psi_test, theta_test, 1)
    println("   rzphi values:")
    println("     f[1] (r²)     = $(round(rzphi_result[1], digits=6))")
    println("     f[2] (φ_shift)= $(round(rzphi_result[2], digits=6))")
    println("     f[3] (z_shift)= $(round(rzphi_result[3], digits=6))")
    println("     f[4] (jacobian)= $(round(rzphi_result[4], digits=6))")
    
    if length(rzphi_result) >= 8
        println("     fx[1] (∂r²/∂ψ) = $(round(rzphi_result[1+4], digits=6))")
        println("     fx[4] (∂J/∂ψ)  = $(round(rzphi_result[4+4], digits=6))")
    end
    
    if length(rzphi_result) >= 12
        println("     fy[1] (∂r²/∂θ) = $(round(rzphi_result[1+8], digits=6))")
        println("     fy[2] (∂φ/∂θ)  = $(round(rzphi_result[2+8], digits=6))")
    end
catch e
    println("   ❌ rzphi evaluation failed: $e")
end

# Test sq evaluation
try
    sq_result = JPEC.SplinesMod.spline_eval(plasma_eq.sq, psi_test, 1)
    println("   sq values:")
    println("     sq[1] (flux)   = $(round(sq_result[1], digits=6))")
    println("     sq[2] (pressure)= $(round(sq_result[2], digits=6))")
    println("     sq[3] (f=RBφ)  = $(round(sq_result[3], digits=6))")
    println("     sq[4] (q safety)= $(round(sq_result[4], digits=6))")
    
    if length(sq_result) >= 8
        println("     sq'[2] (p')    = $(round(sq_result[2+4], digits=6))")
        println("     sq'[4] (q')    = $(round(sq_result[4+4], digits=6))")
    end
catch e
    println("   ❌ sq evaluation failed: $e")
end



    # Test metric evaluation
    if metric_result.fspline !== nothing
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   📈 Computed metric components:")
        println("     g11 = $(round(components[1], digits=6))")
        println("     g22 = $(round(components[2], digits=6))")
        println("     g33 = $(round(components[3], digits=6))")
        println("     g23 = $(round(components[4], digits=6))")
        println("     g31 = $(round(components[5], digits=6))")
        println("     g12 = $(round(components[6], digits=6))")
        println("     jac = $(round(components[7], digits=6))")
        println("     jac1 = $(round(components[8], digits=6))")
    end
    
catch e
    println("❌Error: $e")
end

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [ ]:
# Fourfit Matrix Calculation Example
println("🚀 Fourfit Matrix Calculation")
println("="^30)

using .FourfitMetric
using Interpolations


try
    # Step 1: Metric tensor calculation
    println("1️⃣  Calculating Metric Tensor...")
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,
        plasma_eq.sq;
        mpsi=128,
        mtheta=256,
        mband=2,
        fft_flag=false,
        verbose=true
    )
    
    # Step 2: Matrix calculation
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    # Calculate psio (Toroidal flux normalization)
    psio = 2π  # Default value, should be obtained from equilibrium data
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,
        plasma_eq.sq,
        plasma_eq.rzphi,
        psio;
        nn=1,           # Toroidal mode number
        mlow=-50,        # Minimum poloidal mode
        mhigh=50,        # Maximum poloidal mode
        power_flag=true,
        feval_flag=true,
        sas_flag=true
    )
    
    # Step 3: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix dimensions: $(size(matrix_result.amat))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    # Example of eigenvalue computation
    if matrix_result.amat !== nothing
        eigenvals = FourfitMetric.compute_eigenvalues(matrix_result, 0.5)
        if length(eigenvals) > 0
            println("   First 3 eigenvalues:")
            for (i, λ) in enumerate(eigenvals[1:min(3, length(eigenvals))])
                println("     λ[$i] = $(round(real(λ), digits=6)) + $(round(imag(λ), digits=6))i")
            end
        end
    end
    
    println("✅ All calculations completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("\n🔍 Error Analysis:")
    if isa(e, UndefVarError)
        println("   Undefined variable error: $(e.var)")
        println("   This might be a module scope issue.")
    elseif isa(e, MethodError)
        println("   Method error: $(e.f)")
        println("   Argument types: $(typeof.(e.args))")
        println("   This might be an issue with spline evaluation.")
    elseif isa(e, BoundsError)
        println("   Array index out of bounds error")
        println("   Please check the array dimensions.")
    end
    println("\n💡 Suggestions:")
    println("   1. Ensure equilibrium data is loaded correctly.")
    println("   2. Check the return format of spline_eval.")
    println("   3. Verify all variables are defined in the correct scope.")
end


In [ ]:
#=
======================================================================
4. VALIDATION FOR 2D FOURIER SPLINE (CUBIC-X, FOURIER-Y)
======================================================================
=#
# Assuming FourierSpline is where the module is located
# For standalone testing, you might need:
# include("FourierSpline.jl")
# using .FourierSpline

println("\n--- Testing 2D Fourier-Spline Derivatives ---")



# --- 4.1 Data Generation (for fitting) ---
# For fit_method=2 (FFT), my_nodes must be a power of 2.
# We use mx+1 points for x and my+1 points for y.
# The grid is [0, mx] and [0, my].
mx_nodes, my_nodes = 32, 32 # my_nodes must be a power of 2 for FFT test
x_nodes = collect(range(0.0, stop=3.0, length=mx_nodes + 1))
# y is periodic from 0 to 2pi. The last point y_nodes[my+1] should equal the first y_nodes[1] + 2pi
y_nodes = collect(range(0.0, stop=2*pi, length=my_nodes + 1))

f_nodes = zeros(mx_nodes + 1, my_nodes + 1, 1) # n_qty = 1
for j in 1:(my_nodes + 1), i in 1:(mx_nodes + 1)
    f_nodes[i, j, 1] =  x_nodes[i]^2 * sin(2 * y_nodes[j])
end

# --- 4.2 Spline Setup ---
# Use bctype=1 (natural) for x-direction as it's not periodic.
# The y-direction is inherently periodic.
# mband determines how many Fourier modes are used. A higher number gives more accuracy.
mband_val = 10

# Test both fit methods
println("\n--- Setting up with fit_method=1 (Integral) ---")
fspline_m1 = fspline_setup(x_nodes, y_nodes, f_nodes, mband_val, bctype=4, fit_method=1,fit_flag=true)

println("\n--- Setting up with fit_method=2 (FFT) ---")
# This requires my_nodes to be a power of 2.
fspline_m2 = fspline_setup(x_nodes, y_nodes, f_nodes, mband_val, bctype=2, fit_method=2,fit_flag=true)


In [ ]:

# --- 4.3 Evaluation ---
# We will evaluate using the FFT-fitted spline (method 2) as it's generally faster if applicable.
# You can switch to fspline_m1 to compare.
chosen_fspline = fspline_m1

nx_fine, ny_fine = 60, 60
x_fine = collect(range(0.0, stop=3.0, length=nx_fine))
y_fine = collect(range(0.0, stop=2*pi, length=ny_fine))

# Get derivatives up to 2nd order
f_s, fx_s, fy_s, fxx_s, fxy_s, fyy_s = fspline_eval(chosen_fspline, x_fine, y_fine, Int(2))


# --- 4.4 Analytical (True) Functions (Periodic in y) ---
f_analytic(x, y)   = x^2 * sin(2*y)
fx_analytic(x, y)  = 2*x * sin(2*y)
fy_analytic(x, y)  = -2 * x^2 * cos(2*y)
fxx_analytic(x, y) = 2 * sin(2*y)
fyy_analytic(x, y) = -4 * x^2 * sin(2*y)
fxy_analytic(x, y) = -4 * x * cos(2*y)

print("evaluation complete")